Heaviy adapted from Week5 tutorial

In [1]:

import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import torch

initialisation

In [2]:
device = torch.cuda.current_device() if torch.cuda.is_available() else "cpu" # This line checks if a GPU is available and sets the device to GPU (e.g., cuda:0) or CPU.
print(device)
#device = "cpu"

# Initialise the model and tokenizer to a pre-trained model. Suggestions: facebook/opt-350m, bigscience/bloom-560m

# model is made to classify sentence into 1/3 categories -> wa is a seq classification task
model = AutoModelForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

#model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m").to(device)
#tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

0


Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Data processing

In [3]:
from datasets import Dataset

fp = "../DATASETS/work_arrangements_development_set.csv"
df = pd.read_csv(fp)
df.drop("id", axis=1, inplace=True) #get rid of id column
df.rename(columns={"job_ad": "text"}, inplace=True)

#https://github.com/huggingface/notebooks/blob/main/examples/text_classification-tf.ipynb -> credit
id2label = {
    0:'Hybrid',
    1:'OnSite',
    2:'Remote'
            }
label2id = {val: key for key, val in id2label.items()}

df['label'] = df['y_true'].map(label2id)    #convert label to enum
print(df)



                                                 text  y_true  label
0   Job title: CEO\nAbstract: Exciting opportunity...  Remote      2
1   Job title: Home-Based Online ESL Teacher (Onli...  Remote      2
2   Job title: Safeguarding, De La Salle\nAbstract...  Hybrid      0
3   Job title: Delivery Driver\nAbstract: Pickup t...  OnSite      1
4   Job title: Store Supervisor\nAbstract: We are ...  OnSite      1
..                                                ...     ...    ...
94  Job title: Senior Pipeline Technical Director\...  Hybrid      0
95  Job title: Customer Support Administrator\nAbs...  OnSite      1
96  Job title: Remote Writing Evaluator for AI (As...  Remote      2
97  Job title: People & Culture Advisor\nAbstract:...  Hybrid      0
98  Job title: Draftsperson\nAbstract: Residential...  Hybrid      0

[99 rows x 3 columns]


tokenisation

In [4]:
#tokenised
data = Dataset.from_pandas(df)

data = data.map(lambda samples: tokenizer(samples["text"], truncation=True, padding="max_length", max_length=512), batched=True)
data
data.set_format("torch")    #conversion to pyTorch tensors

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

finetuning

In [5]:
import torch

#GPU sanity check

print(torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name(0))
    print("Memory allocated:", torch.cuda.memory_allocated())

2.6.0+cu118
CUDA Available: True
Device: NVIDIA GeForce GTX 1650 Ti
Memory allocated: 0


In [6]:
import os
training_args = transformers.TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=1,  #keep at 1, otherwise crash
    logging_dir="./logs",
    report_to="none"
)

# === 6. Trainer and training ===
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=data,
    tokenizer=tokenizer
)

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

trainer.train()

/tmp/ipykernel_455/2179141321.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(
Initializing global attention on CLS token...


Step,Training Loss


TrainOutput(global_step=396, training_loss=1.1840026547210385, metrics={'train_runtime': 362.0087, 'train_samples_per_second': 1.094, 'train_steps_per_second': 1.094, 'total_flos': 130057554604032.0, 'train_loss': 1.1840026547210385, 'epoch': 4.0})

Eval

In [7]:
#get test data

fp_test = "../DATASETS/work_arrangements_test_set.csv"
df_test = pd.read_csv(fp_test)
df_test.drop("id", axis=1, inplace=True) #get rid of id column
df_test.rename(columns={"job_ad": "text"}, inplace=True)

df_test


test_data = Dataset.from_pandas(df_test)
testdata = test_data.map(lambda samples: tokenizer(samples["text"], truncation=True, padding="max_length", max_length=512), batched=True)
testdata.set_format("torch")    #conversion to pyTorch tensors

print(testdata)


Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'y_true', 'input_ids', 'attention_mask'],
    num_rows: 99
})


In [ ]:
# Set the model in evaluation mode
model.eval()

correct = 0

print(f"CORRECT: prediction == label")

with torch.no_grad():
    for i in range(len(testdata)):
        sample = testdata[i]
        #print(sample)

        input_ids = sample["input_ids"].unsqueeze(0).to(device)
        attention_mask = sample["attention_mask"].unsqueeze(0).to(device)
        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask)
        
        #print(outputs)
        logits = outputs.logits
        #print(logits)
        pred = torch.argmax(logits, dim=1)  #returns tensor
        predicted_label = id2label[pred.item()]
        if predicted_label == sample['y_true']:
            correct+=1
            print(f"CORRECT: {predicted_label} == {sample['y_true']}")
        else:
            print(f"INCORRECT: {predicted_label} == {sample['y_true']}")

print(f"Accuracy = {correct/len(testdata)}")


CORRECT: prediction == label
CORRECT: OnSite == OnSite
CORRECT: OnSite == OnSite
INCORRECT: OnSite == Remote
INCORRECT: OnSite == Hybrid
INCORRECT: OnSite == Remote
CORRECT: OnSite == OnSite
INCORRECT: OnSite == Hybrid
INCORRECT: OnSite == Remote
INCORRECT: OnSite == Hybrid
INCORRECT: OnSite == Remote
INCORRECT: OnSite == Hybrid
CORRECT: OnSite == OnSite
INCORRECT: OnSite == Remote
CORRECT: OnSite == OnSite
CORRECT: OnSite == OnSite
CORRECT: OnSite == OnSite
CORRECT: OnSite == OnSite
INCORRECT: OnSite == Remote
CORRECT: OnSite == OnSite
INCORRECT: OnSite == Remote
INCORRECT: OnSite == Remote
INCORRECT: OnSite == Hybrid
INCORRECT: OnSite == Remote
INCORRECT: OnSite == Hybrid
CORRECT: OnSite == OnSite
CORRECT: OnSite == OnSite
INCORRECT: OnSite == Hybrid
CORRECT: OnSite == OnSite
INCORRECT: OnSite == Remote
CORRECT: OnSite == OnSite
CORRECT: OnSite == OnSite
INCORRECT: OnSite == Hybrid
INCORRECT: OnSite == Hybrid
INCORRECT: OnSite == Remote
CORRECT: OnSite == OnSite
CORRECT: OnSite == On